# 使用Spark模型实践LangChain功能

In [1]:
import logging
from spark.LLMService.SparkLLM import SparkLLM

# 设置日志级别
logging.basicConfig(level=logging.DEBUG)

llm = SparkLLM()

## 直接调用

In [3]:
llm.invoke("讲一个关于中国足球的笑话")

DEBUG:spark.LLMService.SparkLLM:讲一个关于中国足球的笑话
DEBUG:spark.LLMService.SparkLLM:SparkLLM response: 有一天，中国足球队的教练在训练场上对球员们说：“今天我们要进行一场特殊的训练，那就是闭着眼睛踢球。”

球员们都觉得很好奇，于是纷纷闭上眼睛开始尝试。然而，他们发现这并不容易，因为足球场上有很多障碍物，而且他们还要与队友配合。

过了一会儿，教练看着球员们摸索着踢球的样子，忍不住笑了出来。他问球员们：“你们觉得这个训练有什么意义吗？”

一个球员想了想，回答说：“教练，我觉得这个训练让我们明白了一个道理：要想在足球场上取得成功，光靠眼睛是不够的，我们还需要用心去感受。”

教练听了，满意地点了点头：“没错，足球不仅仅是一项运动，更是一种精神。只要我们用心去踢，总有一天会取得好成绩的。”


'有一天，中国足球队的教练在训练场上对球员们说：“今天我们要进行一场特殊的训练，那就是闭着眼睛踢球。”\n\n球员们都觉得很好奇，于是纷纷闭上眼睛开始尝试。然而，他们发现这并不容易，因为足球场上有很多障碍物，而且他们还要与队友配合。\n\n过了一会儿，教练看着球员们摸索着踢球的样子，忍不住笑了出来。他问球员们：“你们觉得这个训练有什么意义吗？”\n\n一个球员想了想，回答说：“教练，我觉得这个训练让我们明白了一个道理：要想在足球场上取得成功，光靠眼睛是不够的，我们还需要用心去感受。”\n\n教练听了，满意地点了点头：“没错，足球不仅仅是一项运动，更是一种精神。只要我们用心去踢，总有一天会取得好成绩的。”'

## 使用提示词模板

In [17]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(
    input_variables=["joke", "name"],
    template="给我讲一个关于{joke}的笑话",
    validate_template=True
)
prompt = prompt_template.format(joke='中国足球')
llm.invoke(prompt)

DEBUG:spark.LLMService.SparkLLM:给我讲一个关于中国足球的笑话
DEBUG:spark.LLMService.SparkLLM:SparkLLM response: 有一天，一位外国朋友来到中国，他非常好奇地问一位中国人：“你们中国足球的水平怎么样？”

中国人微笑着回答：“我们的足球水平在世界上排名不是很高，但是我们有一项世界领先的技术。”

外国人很好奇地问：“哦？那是什么技术？”

中国人自豪地说：“我们拥有世界上最先进的足球裁判技术！”


'有一天，一位外国朋友来到中国，他非常好奇地问一位中国人：“你们中国足球的水平怎么样？”\n\n中国人微笑着回答：“我们的足球水平在世界上排名不是很高，但是我们有一项世界领先的技术。”\n\n外国人很好奇地问：“哦？那是什么技术？”\n\n中国人自豪地说：“我们拥有世界上最先进的足球裁判技术！”'

In [18]:
template = "给我讲一个关于{joke}的笑话"
prompt_template = PromptTemplate.from_template(template)
prompt = prompt_template.format(joke='中国足球')
# llm.invoke(prompt)

## 序列化提示词模板

In [22]:
from langchain.prompts import load_prompt

# 序列化提示词模板
prompt_template.save("joke_prompt.json")
prompt_template.save("joke_prompt.yaml")
# 反序列化提示词模板
loaded_prompt = load_prompt("joke_prompt.json")
print(loaded_prompt)
loaded_prompt = load_prompt("joke_prompt.yaml")
print(loaded_prompt)

input_variables=['joke'] template='给我讲一个关于{joke}的笑话'
input_variables=['joke'] template='给我讲一个关于{joke}的笑话'


## 具有少量示例的提示词模板

In [25]:
from langchain.prompts import FewShotPromptTemplate

examples = [
    {"word": "高兴", "antonym": "难过"},
    {"word": "高", "antonym": "矮"},
]
example_formatter_template = """
单词: {word}
反义词: {antonym}\n
"""
example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=example_formatter_template,
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="对于每个输入，给出其反义词",
    suffix="单词: {input}\n反义词:",
    input_variables=["input"],
    example_separator=" ",
)
print(few_shot_prompt.format(input='大'))

对于每个输入，给出其反义词 
单词: 高兴
反义词: 难过

 
单词: 高
反义词: 矮

 单词: 大
反义词:


## 输出解析器

In [31]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from langchain.output_parsers import OutputFixingParser



# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

new_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)

misformatted = "{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}"

new_parser.parser(misformatted)

# prompt = PromptTemplate(
#     template="Answer the user query.\n{format_instructions}\n{query}\n",
#     input_variables=["query"],
#     partial_variables={"format_instructions": parser.get_format_instructions()}
# )
#
# # And a query intented to prompt a language model to populate the data structure.
# joke_query = "Tell me a joke."
# _input = prompt.format_prompt(query=joke_query)
# output = llm.invoke(_input.to_string())
# parser.parse(output)

TypeError: 'PydanticOutputParser' object is not callable

In [28]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="列出5种{car}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)
_input = prompt.format(car="比亚迪电车")
output = llm.invoke(_input)
output_parser.parse(output)

DEBUG:spark.LLMService.SparkLLM:列出5种比亚迪电车.
Your response should be a list of comma separated values, eg: `foo, bar, baz`
DEBUG:spark.LLMService.SparkLLM:SparkLLM response: 比亚迪唐EV, 比亚迪汉EV, 比亚迪宋Pro EV, 比亚迪秦Pro EV, 比亚迪e2


['比亚迪唐EV', '比亚迪汉EV', '比亚迪宋Pro EV', '比亚迪秦Pro EV', '比亚迪e2']

## 调用链

In [7]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt_template)

In [9]:
chain.invoke("中国足球")

DEBUG:spark.LLMService.SparkLLM:给我讲一个关于中国足球的笑话
DEBUG:spark.LLMService.SparkLLM:SparkLLM response: 有一天，一位外国朋友来到中国，他非常好奇地问中国朋友：“你们中国人为什么喜欢踢足球呢？”

中国朋友回答：“因为我们中国人热爱运动，足球是我们的国球。”

外国朋友又问：“那你们的国家队在国际比赛中表现如何呢？”

中国朋友笑着说：“我们的国家队在国际比赛中表现得非常出色，总是能够给对手带来惊喜。”

外国朋友疑惑地问：“真的吗？那你们的国家队在世界排名是多少呢？”

中国朋友想了想，回答：“我们国家队的世界排名总是在变化，有时候我们会排在第一，有时候我们会排在第二，还有时候我们会排在第三。”

外国朋友更加疑惑了：“那到底你们国家队的世界排名是多少呢？”

中国朋友微笑着说：“其实，我们国家队的世界排名并不重要，重要的是我们永远热爱足球，永远支持我们的国家队。”


{'joke': '中国足球',
 'text': '有一天，一位外国朋友来到中国，他非常好奇地问中国朋友：“你们中国人为什么喜欢踢足球呢？”\n\n中国朋友回答：“因为我们中国人热爱运动，足球是我们的国球。”\n\n外国朋友又问：“那你们的国家队在国际比赛中表现如何呢？”\n\n中国朋友笑着说：“我们的国家队在国际比赛中表现得非常出色，总是能够给对手带来惊喜。”\n\n外国朋友疑惑地问：“真的吗？那你们的国家队在世界排名是多少呢？”\n\n中国朋友想了想，回答：“我们国家队的世界排名总是在变化，有时候我们会排在第一，有时候我们会排在第二，还有时候我们会排在第三。”\n\n外国朋友更加疑惑了：“那到底你们国家队的世界排名是多少呢？”\n\n中国朋友微笑着说：“其实，我们国家队的世界排名并不重要，重要的是我们永远热爱足球，永远支持我们的国家队。”'}

In [1]:
%pip install pydantic

Note: you may need to restart the kernel to use updated packages.
